In [1]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [2]:
import pathfinding

In [3]:
import sys

import matplotlib.pyplot as plt

sys.path.append('../scripts')
sys.path.append('../')

from heapq import heappop, heappush
from itertools import count

import time
from multiprocessing import Pool
from time import sleep

import numpy as np
from tqdm.notebook import trange, tqdm
from scripts import centroids_graph_builder, graph_osm_loader, utils, clustering

import networkx as nx
import leidenalg as la
import pandas as pd

from joblib import Parallel, delayed

In [4]:
def get_graph():
    dataset=pd.read_csv('../data/graph_data/p2p-Gnutella31.txt',delimiter="\t")
    g = nx.DiGraph()
    nodes = set()
    edges = set()
    for _,r in tqdm(dataset.iterrows(),total=len(dataset.index), desc = 'read data'):
        f,t = r['# FromNodeId'], r['ToNodeId']
        nodes.add(f)
        nodes.add(t)
        edges.add((f,t))
    for n in tqdm(nodes, desc='add nodes'):
        g.add_node(n)
    for u,v in tqdm(edges, desc='add edges'):
        g.add_edge(u,v,length = 1)
    
    del dataset
    return g

In [5]:
def dijkstra_pfa_min_dst(graph: nx.Graph,
                 start: set[int],
                 ) -> \
        tuple[float, list[int]]:
    adjacency = graph._adj
    c = count()
    push = heappush
    pop = heappop
    dist = {}
    fringe = []
    for s in start:
        dist[s] = 0.0
        push(fringe, (0.0, next(c), s))
    while fringe:
        (d, _, v) = pop(fringe)
        for u, e in adjacency[v].items():
            vu_dist = d + e['length']
            if u not in dist or dist[u] > vu_dist:
                dist[u] = vu_dist
                push(fringe, (vu_dist, next(c), u))
    return dist


In [6]:
def get_rand_graph(N, p):
    G = nx.fast_gnp_random_graph(N, p, directed=False)
    if not nx.is_connected(G):
        tmp = []
        for n in nx.connected_components(G):
            for q in n:
                tmp.append(q)
                break
        for i in range(len(tmp) - 1):
            G.add_edge(tmp[i], tmp[i + 1])
    for e in G.edges:
        G.add_edge(e[0], e[1], length=np.random.random_sample() + 0.001)
    for u in G.nodes:
        if u in G[u]:
            G.remove_edge(u, u)
    return G

In [7]:
GRAPH_ID = 'R2555133'  # R13470549 R2555133 R3766483
# примеры id есть в graph_osm_loader.py
# g = get_rand_graph(1000, 0.1)  # загрузка графа
N = 100000
p = 3/N
# g = get_rand_grAaph(N,p)
# g = get_graph()
g= graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))

17883 26972


In [8]:
g.remove_edges_from(nx.selfloop_edges(g))

In [9]:
cms = clustering.resolve_k_means_communities(g, resolution=400, max_iteration=100, cluster_name='cluster1', print_log=True)
print(len(cms))

2024-11-24 12:15:25,129 - root - INFO - communities: 3107


100%|██████████| 100/100 [00:06<00:00, 16.00it/s]

3107


In [10]:
# cms = clustering.leiden(g, partition_type=la.CPMVertexPartition, resolution_parameter = 0.00001, weights='length')
# len(cms), len(g.nodes)

In [11]:
cls2n = centroids_graph_builder.get_cls2n(g, name='cluster1')  # мапа кластер к соседним кластерам 
g1, cls2c = centroids_graph_builder.build_center_graph(g, cms, cls2n, log=True, name='cluster1')

find edges: 100%|██████████| 3107/3107 [00:00<00:00, 5779.79it/s]


In [15]:
points = [utils.get_node_for_initial_graph_v2(g) for _ in trange(1000, desc='generate points')]

generate points:   0%|          | 0/1000 [00:00<?, ?it/s]

In [16]:
d_clusters = np.zeros((len(cms), len(cms)))

d_clusters.size * d_clusters.itemsize / 1024 / 1024

73.64997100830078

In [17]:
import pickle


def calc(data):
    i,points, name, cms, g = data 
    d_clusters = np.zeros((len(cms), len(cms)))

    for u in points:
        ll = dijkstra_pfa_min_dst(g, cms[u])
        q = {}
        for v, d in ll.items():
            if g.nodes()[v][name] in q:
                q[g.nodes()[v][name]] = min(q[g.nodes()[v][name]], d)
            else:
                q[g.nodes()[v][name]] = d
        for v in range(len(cms)):
            if v in q:
                d_clusters[u, v] = q[v]
    with open(f'./temp/{i}.pcl', 'wb') as fp:
        pickle.dump(d_clusters, fp)
    del  i,points, name, cms, g, d_clusters
    return 0

In [18]:
w = 10

In [19]:
from threading import Semaphore

def generator_data():
    s = Semaphore(w * 2)
    for p in range(len(cms)):
        s.acquire()
        yield ({p}, 'cluster1', cms, g)
        s.release()

In [20]:
cms_points = list(range(len(cms)))
data = [(i, cms_points[i::w],'cluster1', cms, g) for i in range(w)]

In [21]:
! mkdir temp 

In [22]:
with Pool(w) as p:
    res = list(tqdm(p.imap_unordered(calc, data), total = len(data)))
for i in range(w):
    with open(f'./temp/{i}.pcl', 'rb') as fp:
        d_clusters += pickle.load(fp)

  0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
!rm -rf temp

In [24]:
d_clusters1= d_clusters
nodes = g.nodes()

In [25]:
d_clusters = {}
d_nodes = {}
for u in tqdm(g1.nodes()):
    dst = dijkstra_pfa_min_dst(g, {cls2c[u]})
    for v in g1.nodes():
        d_clusters[u, v] = dst[cls2c[v]]
        d_clusters[v, u] = dst[cls2c[v]]

for u, d in tqdm(g.nodes(data=True)):
    c = cls2c[d['cluster1']]
    l = nx.single_source_dijkstra(g, u, c, weight='length')[0]
    d_nodes[u] = l

  0%|          | 0/3107 [00:00<?, ?it/s]

  0%|          | 0/17883 [00:00<?, ?it/s]

In [26]:
d_clusters1.size * d_clusters1.itemsize / 1024 / 1024

73.64997100830078

In [27]:
# p2remove = set()
# for p1,p2 in tqdm(points):
#     try:
#         nx.single_source_dijkstra(g,p1,p2,weight='length')
#     except Exception as e:
#         print(e)
#         p2remove.add((p1,p2))

In [28]:
# len(p2remove)

In [29]:
# p = []
# for p1,p2 in points:
#     if (p1,p2)not  in p2remove:
#         p.append((p1,p2))
# points = p

In [30]:


def dijkstra_pfa(graph: nx.Graph,
                 start: int,
                 end: int,
                 cms: set[int] | None = None) -> \
        tuple[float, list[int]]:
    if start == end:
        return 0, [start]
    adjacency = graph._adj
    nodes = graph.nodes()
    c = count()
    push = heappush
    pop = heappop
    dist = {}
    pred = {}
    fringe = []
    push(fringe, (0.0, next(c), 0, start, None))
    while fringe:
        (d, _, n, v, p) = pop(fringe)
        if v in dist:
            continue
        dist[v] = (d, n)
        pred[v] = p
        if v == end:
            break
        for u, e in adjacency[v].items():
            if cms and nodes[u]['cluster'] not in cms:
                continue
            vu_dist = d + e['length']
            if u not in dist:
                push(fringe, (vu_dist, next(c), n + 1, u, v))
    d, n = dist[end]
    n += 1
    path = [None] * n
    i = n - 1
    e = end
    while i >= 0:
        path[i] = e
        i -= 1
        e = pred[e]
    return d, path, set(dist.keys())


def bi_dijkstra_pfa(graph: nx.Graph,
                    start: int,
                    end: int,
                    cms: set[int] | None = None
                    ) -> tuple[float, list[int]]:
    if start == end:
        return 0, [start]
    push = heappush
    pop = heappop
    dist = ({start: (0, 0, None)}, {end: (0, 0, None)})
    fringe = ([], [])
    c = count()

    adjacency = graph._adj
    nodes = graph.nodes()

    push(fringe[0], (0, next(c), 0, start))
    push(fringe[1], (0, next(c), 0, end))

    union_node = None
    union_dst = float('inf')
    while fringe[0] and fringe[1]:
        (d1, _, n1, v1) = pop(fringe[0])
        (d2, _, n2, v2) = pop(fringe[1])
        for u, e in adjacency[v1].items():
            if cms and nodes[u]['cluster'] not in cms:
                continue

            vu_dist = d1 + e['length']
            if u not in dist[0] or dist[0][u][0] > vu_dist:
                dist[0][u] = (vu_dist, n1 + 1, v1)
                push(fringe[0], (vu_dist, next(c), n1 + 1, u))
            if u in dist[1]:
                dd = dist[1][u][0] + dist[0][u][0]
                if dd < union_dst:
                    union_dst = dd
                    union_node = u
        for u, e in adjacency[v2].items():
            if cms and nodes[u]['cluster'] not in cms:
                continue

            vu_dist = d2 + e['length']
            if u not in dist[1] or dist[1][u][0] > vu_dist:
                dist[1][u] = (vu_dist, n2 + 1, v2)
                push(fringe[1], (vu_dist, next(c), n2 + 1, u))
            if u in dist[0]:
                dd = dist[0][u][0] + dist[1][u][0]
                if dd < union_dst:
                    union_dst = dd
                    union_node = u
        if d1 + d2 > union_dst:
            break

    d1, n1, _ = dist[0][union_node]
    d2, n2, _ = dist[1][union_node]
    path = [0] * (n1 + n2 + 1)
    e = union_node
    i = n1
    while dist[0][e][2] is not None:
        path[i] = e
        i -= 1
        e = dist[0][e][2]
    path[0] = e

    e = union_node
    i = n1
    while dist[1][e][2] is not None:
        path[i] = e
        i += 1
        e = dist[1][e][2]
    path[-1] = e
    return union_dst, path, set(dist[0].keys()).union(set(dist[1].keys()))


In [31]:
d_nodes1 = np.zeros((len(g.nodes)))
d_clusters2 = np.zeros(d_clusters1.shape)

In [32]:
for i,j in d_clusters:
    d_clusters2[i,j] = d_clusters[i,j]
d_clusters = d_clusters2

In [90]:
for i,(u,d) in enumerate(g.nodes(data=True)):
    d['n'] = i
    d_nodes1[i] = d_nodes[u]
d_nodes = d_nodes1

In [202]:
def func(u, v):
    n1,n2 = nodes[u],nodes[v]
    c1 = n1['cluster1']
    c2 = n2['cluster1']   
    n1 = n1['n']
    n2 = n2['n']   
    d1 = d_nodes[n1] - d_nodes[n2] - d_clusters[c1,c2]
    d2 = abs(d_clusters[c1,c2] - d_nodes[n1]) - d_nodes[n2] 
    d3 = abs(d_clusters[c1,c2] - d_nodes[n2]) - d_nodes[n1]
    d4 = d_nodes[n2] - d_nodes[n1] - d_clusters[c1,c2]
    # print(d_clusters1[c1,c2], d2,d3)
    return np.max(d_clusters1[c1, c2],d2,d3,d1,d4)

In [194]:
def dijkstra_pfa_cls(graph: nx.Graph,
                     start: int,
                     end: int
                     ) -> tuple[float, list[int]]:
    if start == end:
        return 0, [start]
    push = heappush
    pop = heappop
    dist = {start: (0, None)}
    fringe = []
    c = count()
    adjacency = graph._adj
    push(fringe, (0, next(c), 0, func(start, end), start))

    while fringe:
        (_, _, d1, l1, v1) = pop(fringe) 
        if v1 == end:
            break
        for u, e in adjacency[v1].items():
            vu_dist = d1 + e['length']
            if u not in dist or dist[u][0] > vu_dist:
                dist[u] = (vu_dist, v1)
                lu = func(u, end)
                push(fringe, (vu_dist + lu, next(c), vu_dist, lu, u))

    path = [end]
    e = end
    while dist[e][1] is not None:
        e = dist[e][1]
        path = [e] + path
    l = 0
    e = g.edges()
    for i in range(len(path) - 1):
        p1, p2 = path[i], path[i + 1]
        l += e[p1, p2]['length']
    return l, path, set(dist.keys())

In [195]:
def ch_pfa(
        graph: nx.Graph,
        start: int,
        end: int) -> tuple[float, list[int]]:
    if start == end:
        return 0, [start]
    adjacency = graph._adj
    push = heappush
    pop = heappop
    dist = (set(), set())
    fringe = ([], [])
    c = count()

    push(fringe[0], (0, next(c), 0, start))
    push(fringe[1], (0, next(c), 0, end))

    heads = [0, 0]
    seens = ({start: (0, None)}, {end: (0, None)})
    union_node = None
    union_dst = float('inf')
    dir = 1
    while fringe[0] or fringe[1]:
        if fringe[0] and fringe[1]:
            dir = 1 - dir
        elif fringe[0]:
            dir = 0
        else:
            dir = 1

        (_, _, d, v) = pop(fringe[dir])

        heads[dir] = d

        if v in dist[dir]:
            continue

        dist[dir].add(v)

        for u, l in adjacency[v].items():
            vu_dist = d + l['length']
            if u not in dist[dir] and (u not in seens[dir] or seens[dir][u][0] > vu_dist):
                seens[dir][u] = (vu_dist, v)
                l = -(func(start, u) -  func(end, u))/2 if dir == 1 else (func(start, u) -  func(end, u))/2
                push(fringe[dir], (vu_dist + l, next(c),vu_dist, u))
                if u in seens[1 - dir]:
                    tpl = seens[1 - dir][u]
                    dd = tpl[0] + vu_dist
                    if dd < union_dst:
                        union_dst = dd
                        union_node = u
        if min(heads) > union_dst:
            break
    path = []

    e = union_node
    while seens[0][e][1] is not None:
        path = [e] + path
        e = seens[0][e][1]
    path = [e] + path

    e = union_node
    while seens[1][e][1] is not None:
        e = seens[1][e][1]
        path += [e]

    l = 0
    e = g.edges()
    for i in range(len(path) - 1):
        p1, p2 = path[i], path[i + 1]
        l += e[p1, p2]['length']
    return l, path


In [196]:
def bi_dijkstra_pfa_cls(graph: nx.Graph,
                        start: int,
                        end: int
                        ) -> tuple[float, list[int]]:
    if start == end:
        return 0, [start]
    push = heappush
    pop = heappop
    dist = ({start: (0, None)}, {end: (0, None)})
    fringe = ([], [])
    c = count()
    adjacency = graph._adj

    push(fringe[0], (0, next(c), 0, 0, start))
    push(fringe[1], (0, next(c), 0, 0, end))

    union_node = None
    union_dst = float('inf')
    while fringe[0] and fringe[1]:
        (_, _, d1, l1, v1) = pop(fringe[0])
        (_, _, d2, l2, v2) = pop(fringe[1])

        for u, e in adjacency[v1].items():
            vu_dist = d1 + e['length']
            if u not in dist[0] or dist[0][u][0] > vu_dist:
                dist[0][u] = (vu_dist, v1)
                lu = -(func(u, start) - func(u,end))/2
                push(fringe[0], (vu_dist + lu, next(c), vu_dist, lu, u))
                if u in dist[1]:
                    dd = dist[1][u][0] + dist[0][u][0]
                    if dd < union_dst:
                        union_dst = dd
                        union_node = u

        for u, e in adjacency[v2].items():
            vu_dist = d2 + e['length']
            if u not in dist[1] or dist[1][u][0] > vu_dist:
                dist[1][u] = (vu_dist, v2)
                lu = (func(u, start) - func(u,end))/2
                push(fringe[1], (vu_dist + lu, next(c), vu_dist, lu, u))
                if u in dist[0]:
                    dd = dist[0][u][0] + dist[1][u][0]
                    if dd < union_dst:
                        union_dst = dd
                        union_node = u
        if d1 + d2 > union_dst:
            break
    d1, _ = dist[0][union_node]
    d2, _ = dist[1][union_node]
    path = []

    e = union_node
    while dist[0][e][1] is not None:
        path = [e] + path
        e = dist[0][e][1]
    path = [e] + path

    e = union_node
    while dist[1][e][1] is not None:
        e = dist[1][e][1]
        path += [e]

    l = 0
    e = g.edges()
    for i in range(len(path) - 1):
        p1, p2 = path[i], path[i + 1]
        l += e[p1, p2]['length']
    return l, path

In [180]:
pos = {u: [d['x'], d['y']] for u, d in g.nodes(data=True)}


In [181]:
p1, p2 = points[4]

In [192]:
dijkstra_pfa_cls(g,p1,p2)

2916.4949999999994 3194.9530000000004 3194.9530000000004
2916.4949999999994 3206.811 3206.811
2916.4949999999994 3158.6220000000003 3158.6220000000003
2916.4949999999994 3234.2470000000003 3234.2470000000003
3388.726 3608.9540000000006 3608.9540000000006
2924.662 3166.789 3166.789
2385.917 2920.273 2920.273
2385.917 3013.63 3013.63
2385.917 2976.5860000000002 2976.5860000000002
2385.917 2771.406 2771.406
2385.917 2622.1130000000003 2622.1130000000003
2385.917 2912.877 2912.877
2385.917 2917.4120000000003 2917.4120000000003
2305.328 2749.956 2749.956
3105.351 3347.4780000000005 3347.4780000000005
2305.328 2559.0950000000003 2559.0950000000003
2305.328 2547.455 2547.455
2385.917 2721.495 2721.495
3388.726 3551.9940000000006 3551.9940000000006
3461.632 3703.7590000000005 3703.7590000000005
3105.351 3175.2210000000005 3175.2210000000005
2305.328 2422.3410000000003 2422.3410000000003
2906.6209999999996 3148.7480000000005 3148.7480000000005
1840.3609999999999 2083.689 2083.689
1840.360999999

(3194.9529999999995,
 [249544760,
  690915721,
  347447440,
  267391897,
  8311547252,
  8311547250,
  260328156,
  6950556447,
  7648865942,
  347962521,
  9128626709,
  7648859472,
  3556341874,
  3917371583,
  4974239207,
  8416859194,
  8738859109,
  8044993459,
  8044993460,
  8044993480],
 {249378501,
  249544023,
  249544499,
  249544511,
  249544584,
  249544586,
  249544760,
  249689987,
  260328156,
  260328158,
  260328161,
  267380483,
  267391897,
  269167423,
  275104477,
  303586376,
  303586417,
  303586553,
  303586934,
  303587870,
  303587873,
  303587877,
  303588014,
  347447440,
  347447448,
  347962521,
  444816949,
  690915676,
  690915721,
  690976165,
  1003819252,
  1005315429,
  1288826270,
  1886870754,
  3052006166,
  3556341874,
  3556341888,
  3763031286,
  3914345567,
  3915382214,
  3917371548,
  3917371578,
  3917371583,
  4118651614,
  4974239205,
  4974239207,
  5260031318,
  5840577687,
  6048405427,
  6144636723,
  6144636850,
  6950556447,
  6991

In [182]:
# f, ax = plt.subplots(1, 1, figsize=(20, 10))
# l, path, nn = dijkstra_pfa(g, p1, p2)
# f.suptitle('classical')
# nx.draw(g, pos=pos, ax=ax, node_size=10, alpha=0.5)
# nx.draw(g.subgraph(nn), pos=pos, ax=ax, node_size=10, node_color='green')
# nx.draw(g.subgraph(path), pos=pos, ax=ax, node_size=15, node_color='red', edge_color='red', width=2)
# nx.draw(g.subgraph(p1), pos=pos, ax=ax, node_size=30, node_color='blue', edge_color='red', width=2)
# nx.draw(g.subgraph(p2), pos=pos, ax=ax, node_size=30, node_color='black', edge_color='red', width=2)
# # plt.show()

In [183]:
# f, ax = plt.subplots(1, 1, figsize=(20, 10))
# l, path, nn = bi_dijkstra_pfa(g, p1, p2)
# f.suptitle('bi-directional')
# nx.draw(g, pos=pos, ax=ax, node_size=10, alpha=0.5)
# nx.draw(g.subgraph(nn), pos=pos, ax=ax, node_size=15, node_color='olivedrab', width=2)
# nx.draw(g.subgraph(path), pos=pos, ax=ax, node_size=15, node_color='red', edge_color='red', width=3)
# nx.draw(g.subgraph(p1), pos=pos, ax=ax, node_size=30, node_color='blue', edge_color='red', width=3)
# nx.draw(g.subgraph(p2), pos=pos, ax=ax, node_size=30, node_color='black', edge_color='red', width=3)
# # plt.show()

In [184]:
# f, ax = plt.subplots(1, 1, figsize=(20, 10))
# l, path, nn = dijkstra_pfa_cls(g, p1, p2)
# 
# f.suptitle('bi-dir-cluster-heuristic')
# nx.draw(g, pos=pos, ax=ax, node_size=10, alpha=0.5)
# nx.draw(g.subgraph(nn), pos=pos, ax=ax, node_size=15, node_color='olivedrab', width=2)
# nx.draw(g.subgraph(path), pos=pos, ax=ax, node_size=15, node_color='red', edge_color='red', width=3)
# nx.draw(g.subgraph(p1), pos=pos, ax=ax, node_size=30, node_color='blue', edge_color='red', width=3)
# nx.draw(g.subgraph(p2), pos=pos, ax=ax, node_size=30, node_color='black', edge_color='red', width=3)
# # plt.show()

In [185]:
# f, ax = plt.subplots(1, 1, figsize=(20, 10))
# l,path,nn = ch_pfa(chg, p1,p2)
# f.suptitle('cluster-heuristic')
# nx.draw(g,pos=pos, ax = ax, node_size = 10, alpha=0.5)
# nx.draw(g.subgraph(nn),pos=pos, ax = ax, node_size = 15, node_color='olivedrab',width = 2)
# nx.draw(g.subgraph(path),pos=pos, ax = ax, node_size = 15, node_color='red', edge_color='red',width = 3)
# nx.draw(g.subgraph(p1),pos=pos, ax = ax, node_size = 30, node_color='blue', edge_color='red',width = 3)
# nx.draw(g.subgraph(p2),pos=pos, ax = ax, node_size = 30, node_color='black', edge_color='red',width = 3)
# # plt.show()

In [186]:
# for p1,p2 in points:
    # print(func(p1, p2), nx.single_source_dijkstra(g, p1, p2, weight='length')[0])    

In [187]:
p1, p2 = points[2]

In [203]:
NUM_ITERATION = 3  # чтобы уменьшить ошибку при вычислении времени выполнения, при каждом замере время меряется для NUM_ITERATION повторений
WORKER = 8  # количество потоков


def do_calc(data):
    pps, i = data

    stat = {}
    stat['l'] = []
    stat['h_l'] = []

    stat['p'] = []
    stat['h_p'] = []
    stat['time_l'] = []
    stat['time_h'] = []

    stat['delta'] = []
    # чисто чтобы tqdm нормально прогрузился 
    sleep(i / 10)
    print('start', i)

    for p1, p2 in tqdm(pps, desc='find paths', position=i):
        if (p1, p2) in stat:
            continue
        # класический дейкстра
        l, p = None, None
        start = time.time()
        for i in range(NUM_ITERATION):
            l, p = nx.single_source_dijkstra(g, p1, p2, weight='length')
        time_l = time.time() - start

        # иерархический
        h_l, h_p = None, None
        start = time.time()
        for _ in range(NUM_ITERATION):
            h_l, h_p, _ = dijkstra_pfa_cls(g, p1, p2)  #find_path_length_h(g, g1,cms, p1, p2)
            # h_l, h_p,_ = dijkstra_pfa_cls(g, p1, p2)  #find_path_length_h(g, g1,cms, p1, p2)
            # cls = set([g.nodes()[u]['cluster'] for u in h_p])
            # cls1 = cls.copy()
            # for c in cls:
            #     for cc in cls2n[c]:
            #         cls1.add(cc)
            # h_l, h_p = dijkstra.dijkstra_pfa(g, p1,p2, cls1)  #find_path_length_h(g, g1,cms, p1, p2)
            # h_l,h_p=ch_pfa(chg, p1,p2)
            # h_l, h_p = nx.astar_path_length(g, p1,p2,weight='length', heuristic=func), []
        time_h = time.time() - start

        delta = (h_l - l) / l * 100
        stat['l'].append(l)  # длина обычного пути
        stat['h_l'].append(h_l)  # длина иерархического пути
        stat['p'].append(p)  # обычный путь
        stat['h_p'].append(h_p)  # иерархический путь
        stat['delta'].append(delta)  # разница в длине
        stat['time_l'].append(time_l)  # обычное время 
        stat['time_h'].append(time_h)  # иерархическое
    return stat

data = [([p for p in points[i::WORKER]], i) for i in range(WORKER)]
# do_calc(data[0])
with Pool(WORKER) as p:
    res = p.map(do_calc, data)

stat = {}
for l in res:

    for d in l:
        if d not in stat:
            stat[d] = []
        stat[d].extend(l[d])
print(f"err_mean: {np.mean(stat['delta']):.2f} %")
print(f"err_min: {np.min(stat['delta']):.2f} %")
print(f"err_max: {np.max(stat['delta']):.2f} %", )
print(f"acceleration: {np.mean(np.array(stat['time_l']) / np.array(stat['time_h'])):.2f} times")

start 0


find paths:   0%|          | 0/125 [00:00<?, ?it/s]

start 1


find paths:   0%|          | 0/125 [00:00<?, ?it/s]

start 2


...

start 6


find paths:   0%|          | 0/125 [00:00<?, ?it/s]

start 7


find paths:   0%|          | 0/125 [00:00<?, ?it/s]

err_mean: 0.00 %
err_min: 0.00 %
err_max: 0.00 %
acceleration: 35.82 times


In [ ]:
max_err_idx = np.argmax(stat['delta'])


In [ ]:
path = stat['p'][max_err_idx]
h_path = stat['h_p'][max_err_idx]

In [ ]:
path

In [ ]:
h_path

In [ ]:
len(path), len(h_path)

In [ ]:
print(np.array(path) - np.array(h_path))

In [ ]:
stat['l'][max_err_idx], stat['h_l'][max_err_idx]

In [ ]:
p1, p2 = path[0], path[-1]

In [ ]:
labels = {u: d['cluster'] for u, d in g.nodes(data=True)}

In [ ]:
pos_l = {u: [d['x'] + 0.001, d['y'] + 0.001] for u, d in g.nodes(data=True)}
pos = {u: [d['x'], d['y']] for u, d in g.nodes(data=True)}


In [ ]:
f, ax = plt.subplots(1, 1, figsize=(20, 10))
f.suptitle('max_err')
cls = set([g.nodes()[u]['cluster'] for u in h_path])
cls1 = cls.copy()
for c in cls:
    for cc in cls2n[c]:
        cls1.add(cc)
cls = set([g.nodes()[u]['cluster'] for u in path])
for c in cls:
    cls1.add(c)
    for cc in cls2n[c]:
        cls1.add(cc)

nx.draw(centroids_graph_builder.extract_cluster_list_subgraph(g, cls1, cms2), pos=pos, ax=ax, node_size=10, alpha=0.5)
nx.draw(g.subgraph(path), pos=pos, ax=ax, node_size=15, node_color='red', width=2, alpha=0.5)
nx.draw(g.subgraph(h_path), pos=pos, ax=ax, node_size=15, node_color='green', width=2, alpha=0.5)
# nx.draw_networkx_labels(g,pos_l,labels=labels,font_size=6)

In [ ]:
nx.single_source_dijkstra(g,p1,p2,weight='length')

In [ ]:
bi_dijkstra_pfa_cls(g, p1,p2)

In [ ]:
893305603 in nx.single_source_dijkstra(g,p1,p2,weight='length')[1]